# 第六章 使用一等函数实现设计模式

In [2]:
from abc import ABC, abstractmethod
from collections import namedtuple

In [3]:
Customer = namedtuple('Customer', 'name fidelity')

In [4]:
class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.price * self.quantity

In [ ]:
class Order:
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        # prootion此时输入的是个类的对象，可以改成函数
        self.promotion = promotion
    
    def total(self):
        if not hasattr(self, '_total'):
            self._total = sum(item.total() for item in self.cart)
        return self._total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [ ]:
class Promotion(ABC):
    @abstractmethod
    def discount(self, order):
        """return discount"""

class FidelityPromo(Promotion):
    def discount(self, order):
        return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

class BulkItemPromo(Promotion):
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * 0.1
        return discount

class LargeOrderPromo(Promotion):
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * 0.07
        return 0

In [ ]:
# 找出模块中的全部策略
# globals() 返回一个字典，表示当前全局符号
promos = [globals()[name] for name in globals() if name.endswith('_promo') and name != 'best_promo']